In [ ]:
#compose the train and test set
#grandstaff_dataset/<xyz>.krn, grandstaff_dataset/<xyz>.jpg
#'Data/grandstaff_dataset/partitions/train.txt' - where the train txt is that contains ^ file paths

In [3]:
#unzip the tgz raw data file.
#note: change the below string paths as appropriate.

!tar -xzvf "grandstaff.tgz" -C "Data/grandstaff_dataset/"

Streaming output truncated to the last 5000 lines.
scarlatti-d/keyboard-sonatas/L304K470/original_m-123-126_distorted.jpg
scarlatti-d/keyboard-sonatas/L304K470/maj3_down_m-99-102.krn
scarlatti-d/keyboard-sonatas/L304K470/maj2_up_m-75-78_distorted.jpg
scarlatti-d/keyboard-sonatas/L304K470/min3_up_m-144-147_distorted.jpg
scarlatti-d/keyboard-sonatas/L304K470/maj2_down_m-60-64.bekrn
scarlatti-d/keyboard-sonatas/L304K470/maj3_up_m-145-150.jpg
scarlatti-d/keyboard-sonatas/L304K470/original_m-6-9.bekrn
scarlatti-d/keyboard-sonatas/L304K470/maj2_down_m-68-72.krn
scarlatti-d/keyboard-sonatas/L304K470/maj3_up_m-65-70.bekrn
scarlatti-d/keyboard-sonatas/L304K470/min3_up_m-27-30.bekrn
scarlatti-d/keyboard-sonatas/L304K470/maj3_up_m-140-145.bekrn
scarlatti-d/keyboard-sonatas/L304K470/maj2_down_m-116-120.bekrn
scarlatti-d/keyboard-sonatas/L304K470/min3_down_m-0-5.krn
scarlatti-d/keyboard-sonatas/L304K470/min3_up_m-6-9_distorted.jpg
scarlatti-d/keyboard-sonatas/L304K470/maj2_down_m-80-84.jpg
scarlatt

### Create the train, test, and val files

In [ ]:
#let's get the total number of samples from the dataset first
import os

sample_bekrn = [] #here, we're only interested in the bekern files (vs the krn files)

for root, dirs, files in os.walk("./grandstaff_dataset/"):
   for name in files:
      if 'bekrn' in name:
        print(os.path.join(root, name))
        sample_bekrn.append(os.path.join(root, name))

In [ ]:
#let's split the train-val-test sets such that each are 60%, 20%, and 20% of all the samples we have
train_pt, val_pt = 53882*0.6, 53882*0.8
train_pt, val_pt

(32329.199999999997, 43105.600000000006)

In [ ]:
#do the split and double check that it works as expected
train, val, test = sample_bekrn[:32329], sample_bekrn[32329:43106], sample_bekrn[43106:]
len(train), len(val), len(test)

(32329, 10777, 10776)

In [ ]:
#save the different dataset records to text files
def write_testset_files(data, file_name):
    with open(f'{file_name}', 'w') as f:
        for samp in data:
            f.write(f"{samp}\n")

In [ ]:
write_testset_files(train, 'train.txt')
write_testset_files(test, 'test.txt')
write_testset_files(val, 'val.txt')

### Create Dataset datasets

In [16]:
"""Note: In this section, we borrow/adapt code from https://github.com/multiscore/e2e-pianoform, the original source code from the authors that designed and created the raw dataset"""

'Note: In this section, we borrow/adapt code from https://github.com/multiscore/e2e-pianoform, the original source code from the authors that designed and created the raw dataset'

In [6]:
!pip install loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.4 MB/s eta 0:00:00


In [7]:
from torch.utils.data import DataLoader
from data import load_dataset, batch_preparation_ctc, check_and_retrieveVocabulary, PoliphonicDataset

train_path = "Data/grandstaff_dataset/train.txt"
val_path = "Data/grandstaff_dataset/val.txt"
test_path = "Data/grandstaff_dataset/test.txt"

train_dataset = PoliphonicDataset(partition_file=train_path)
val_dataset = PoliphonicDataset(partition_file=val_path)
test_dataset = PoliphonicDataset(partition_file=test_path)


Output()

Output()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Output()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
def make_vocabulary(YSequences, pathToSave, nameOfVoc):
    vocabulary = set()
    for samples in YSequences:
        for element in samples:
                vocabulary.update(element)

    print(len(vocabulary))
    #Vocabulary created
    w2i = {symbol:idx+1 for idx,symbol in enumerate(vocabulary)}
    i2w = {idx+1:symbol for idx,symbol in enumerate(vocabulary)}

    w2i['<pad>'] = 0
    i2w[0] = '<pad>'

    return w2i, i2w

In [13]:
import numpy as np
w2i, i2w = make_vocabulary([train_dataset.get_gt(), val_dataset.get_gt(), test_dataset.get_gt()], "vocab", f"testy")

187


In [14]:
train_dataset.set_dictionaries(w2i, i2w)
val_dataset.set_dictionaries(w2i, i2w)
test_dataset.set_dictionaries(w2i, i2w)

train_dataloader = DataLoader(train_dataset, batch_size=1, num_workers=1, collate_fn=batch_preparation_ctc)
val_dataloader = DataLoader(val_dataset, batch_size=1, num_workers=1, collate_fn=batch_preparation_ctc)
test_dataloader = DataLoader(test_dataset, batch_size=1, num_workers=1, collate_fn=batch_preparation_ctc)

In [15]:
next(iter(train_dataloader))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 
[tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])][tensor([117,   4, 117,  38,  60,   4,  92,  38, 101,   4, 101,  38, 126,   4,
        126,  38, 138,   4, 138,  38,  23,   4, 149,  38, 112,  56, 106, 130,
        136, 112,  56,  17,  24, 130,   4, 112,  56, 100, 130,   4,  75,  56,
        184, 165,  38,  56,   4,  56,   4,  95, 139,  86,  38,  56,   4,  56,
          4,  75, 184,  13,  38,  75, 106, 182, 136,  75,  17,  24, 182,   4,
         75, 100, 165, 182,   4,  75, 184,  38,  75, 185,   4,  75,  39,   4,
         75, 185,  38,  75, 185,   4,  75,  67,  24,  13,   4,  75, 185,  38,
        168,   4, 168,   4, 168,  38,  75,  56,  45, 165,   4, 160,  56,   2,
        130,   4, 112,  56,  55, 158, 130, 136, 112,  56, 184, 130,  38,  95,
         17,  24,  86,   4,  56,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)





[tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])] [tensor([117,   4, 117,  38,  60,   4,  92,  38, 101,   4, 101,  38, 126,   4,
        126,  38, 138,   4, 138,  38, 112, 106, 136, 112,  17,  24,   4,  75,
         56, 100, 165, 136,  75,  56,  67,  24,  38,  56,   4,  95,  84,  24,
         43,  86,  38,  75, 185,   4,  75, 100,  13,  38, 112, 185,   4, 112,
        100,  38,  75, 185,   4,  75, 187,  38, 168,   4, 168,  38,  23,   4,
        149,  38, 112,  56,  55, 130, 136, 112,  56, 184,  24, 130,   4, 112,
         56, 187, 130,   4,  75,  56,   2, 165,  38,  56,   4,  56,   4,  95,
         67,  24,  86,  38,  56,   4,  56,   4,  75,   2,  13,  38,  75,  55,
        182, 136,  75, 184,  24, 182,   4,  75, 187, 165, 182,   4,  75,   2,
         38,  75, 185,   4,  75

(tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]]]]),
 tensor([[117.,   4., 117.,  38.,  60.,   4.,  92.,  38., 101.,   4., 101.,  38.,
          126.,   4., 126.,  38., 138.,   4., 138.,  38.,  23.,   4., 149.,  38.,
          112.,  56., 106., 130., 136., 112.,  56.,  17.,  24., 130.,   4., 112.,
           56., 100., 130.,   4.,  75.,  56., 184., 165.,  38.,  56.,   4.,  56.,
            4.,  95., 139.,  86.,  38.,  56.,   4.,  56.,   4.,  75., 184.,  13.,
           38.,  75., 106., 182., 136.,  75.,  17.,  24., 182.,   4.,  75., 100.,
          165., 182.,   4.,  75., 184.,  38.,  75., 185.,   4.,  75.,  39.,   4.,
           75., 185.,  38.,  75., 185.,   4.,  75.,  67.,  24.,  13.,   4.,  75.,
          185.,  38., 168.,   4., 168.,   4., 168.,  38.,  75.,  5

In [19]:
train_dataset[0][0].shape, train_dataset[0][1].shape, train_dataset[0][2], train_dataset[0][3]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(torch.Size([1, 720, 256]), torch.Size([325]), 1440, 325)